In [41]:
from faker import Faker
from datetime import date
import pandas as pd
import numpy as np

In [3]:
fake = Faker()

# First company

In [28]:
n_users = 1000

In [31]:
user_names = [fake.name() for i in range(n_users)]
user_addresses = [fake.address() for i in range(n_users)]
user_reg_dates = [fake.date_between_dates(date(2015,1,1), date(2017,10,14)) for i in range(n_users)]

In [39]:
users = pd.DataFrame({
        'name': user_names,
        'address': user_addresses,
        'registration_date': user_reg_dates
    })\
    .sort_values('registration_date')\
    .reset_index(drop=True)\
    .reset_index()\
    .rename(columns = {'index': 'id'})

In [46]:
users.loc[0, 'registration_date']

datetime.date(2015, 1, 1)

In [44]:
order_user_ids = np.random.choice(range(n_users), size = n_users * 20)

In [48]:
order_dates = [fake.date_between_dates(users.loc[oui, 'registration_date'], date(2017,10,14)) for oui in order_user_ids]

In [50]:
order_costs = [(np.random.randn() + 1) * 1000 for i in range(len(order_user_ids))]

In [52]:
order_lats = [fake.latitude() for i in range(len(order_user_ids))]
order_lons = [fake.longitude() for i in range(len(order_user_ids))]

In [56]:
orders = pd.DataFrame({
        'date': order_dates,
        'user_id': order_user_ids,
        'lat': order_lats,
        'lon': order_lons,
        'cost': order_costs
    })\
    .sort_values('date')\
    .reset_index(drop=True)\
    .reset_index()\
    .rename(columns = {'index': 'id'})

In [61]:
users.to_csv('data/0_users.csv', index=False)
orders.to_csv('data/0_orders.csv', index=False)

# Parse dates

In [100]:
from dateparser import parse

In [121]:
query = """
How many users registered two days ago
"""

In [192]:
def parse_dates(query):

    words = query.split()

    words_parsed = [0] * len(words)
    for i in range(len(words)):
        for j in range(i, len(words)):
            candidate = ' '.join(words[i:j+1])
            if parse(candidate):
                words_parsed[i:j+1] = [1]*(j+1-i)

    for i in range(len(words)):
        if words[i] in ['and', 'to']:
            words_parsed[i] = 0
                
    parsed_dates = []
    current_date = ''
    for i in range(len(words)+1):
        if i == len(words) or words_parsed[i] == 0:
            if current_date:
                parsed_dates.append(parse(current_date).date())
            current_date = ''
        else:
            current_date = current_date + (' ' if current_date else '')  + words[i]

    parsed_dates = sorted(parsed_dates)
    dates_to_return = []
    if len(parsed_dates) > 0:
        dates_to_return.append(parsed_dates[0])
    if len(parsed_dates) > 1:
        dates_to_return.append(parsed_dates[len(parsed_dates) - 1])
    
    return parsed_dates

In [184]:
q = 'How many users registered last month?'
parse_dates(q)

last month? 2017-09-14 04:11:36.407281


[datetime.date(2017, 9, 14)]

In [185]:
q = 'How many users registered in the last 7 days?'
parse_dates(q)

7 days? 2017-10-07 04:11:48.138956


[datetime.date(2017, 10, 7)]

In [195]:
q = 'How many users registered from 01/09/2017 to 01/10/2017'
parse_dates(q)

[datetime.date(2017, 9, 1), datetime.date(2017, 10, 1)]